In [1]:
!pip install numpy pandas dask tqdm 


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
import re
import numpy as np
import pandas as pd 
from datetime import datetime
from tqdm import tqdm

In [3]:
# display all columns
pd.set_option('display.max_columns', None)

In [4]:
# define basics (Information about Stations (name, lat, long, ids, products, lines)
stations = pd.read_csv('./stations/BVG_complete.csv').set_index('Unnamed: 0')
stations.index.rename("id", inplace=True)
lat_long = stations[['latitude', 'longitude']]
columns = ['stop_id', 'trip_id', 'when', 'planned_when', 'stop_name', 'line_name', 'delay_seconds', 'start_station', 'end_station', 'end_station_id', 'product', 'occupancy', 'remark_summary', 'remark_start', 'remark_end', 'realtimeDataUpdatedAt']

In [5]:
stations.head()

,stop_name,latitude,longitude,product_suburban,product_subway,product_tram,product_bus,product_ferry,product_express,product_regional
id,,,,,,,,,,
900141001,Heinersdorf (Berlin),52.572342,13.437453,False,False,True,False,False,False,False
900141001,Heinersdorf (Berlin),52.572342,13.437453,False,False,True,False,False,False,False
900073101,S Marienfelde (Berlin),52.423813,13.375023,True,False,False,True,False,False,False
900072181,Großbeerenstr./Daimlerstr. (Berlin),52.425872,13.375643,False,False,False,True,False,False,False
900185503,Adlergestell/Vetschauer Allee (Berlin),52.386409,13.633894,False,False,True,True,False,False,False


In [6]:
# Reference date from the file name
reference_date_str = '2023-11-30 17:56:55.799444'
reference_date = datetime.strptime(reference_date_str, '%Y-%m-%d %H:%M:%S.%f')

# Initialize an empty DataFrame to store the combined data
df = pd.DataFrame()
orig_size = 0
# Iterate over the files in the folder
for file in tqdm(os.listdir("./delays")):
    # Check if the file is a CSV file
    if file.endswith('.csv'):
        # Extract the date from the file name
        file_date_str = file.replace('delays', '').replace('.csv', '')
        file_date = datetime.strptime(file_date_str, '%Y-%m-%d %H:%M:%S.%f')
        
        # Check if the file date is later or equal to the reference date
        if file_date >= reference_date:
            # Read the CSV file and append it to the combined DataFrame
            file_path = os.path.join("./delays", file)
            data = open(file_path).readlines()
            data = [re.split(r',(?!\s)', d) for d in data]
            df1 = pd.DataFrame(data, columns=columns)
            orig_size += len(df1)
            df1 = df1.drop_duplicates(subset=['trip_id', 'stop_name'], keep='last').set_index('trip_id')
            df = pd.concat([df, df1])

100%|██████████| 12/12 [05:36<00:00, 28.01s/it]


In [7]:
# Displaying the combined DataFrame
df.head()

,stop_id,when,planned_when,stop_name,line_name,delay_seconds,start_station,end_station,end_station_id,product,occupancy,remark_summary,remark_start,remark_end,realtimeDataUpdatedAt
trip_id,,,,,,,,,,,,,,,
1|44456|0|86|30112023,900141001,2023-11-30T19:03:00+01:00,2023-11-30T19:03:00+01:00,Heinersdorf (Berlin),M2,0,Heinersdorf (Berlin),S+U Alexanderplatz Bhf/Dircksenstr. (Berlin),900100024,tram,low,None,None,None,1701365897\n
1|59997|12|86|30112023,900072181,2023-11-30T19:02:00+01:00,2023-11-30T19:02:00+01:00,Großbeerenstr./Daimlerstr. (Berlin),M77,None,Großbeerenstr./Daimlerstr. (Berlin),Waldsassener Str. (Berlin),900073281,bus,medium,None,None,None,1701365897\n
1|43249|13|86|30112023,900185503,2023-11-30T18:56:00+01:00,2023-11-30T18:56:00+01:00,Adlergestell/Vetschauer Allee (Berlin),68,0,Adlergestell/Vetschauer Allee (Berlin),Alt--Schmöckwitz (Berlin),900185701,tram,low,None,None,None,1701365087\n
1|43266|14|86|30112023,900185503,2023-11-30T19:02:00+01:00,2023-11-30T19:02:00+01:00,Adlergestell/Vetschauer Allee (Berlin),68,None,Adlergestell/Vetschauer Allee (Berlin),Hirtestr. (Berlin),900180016,tram,low,None,None,None,1701365087\n
1|814|34|86|30112023,900100001,None,2023-11-30T19:03:00+01:00,S+U Friedrichstr. Bhf (Berlin),S25,None,S+U Friedrichstr. Bhf (Berlin),S Teltow Stadt,900220114,suburban,None,S25 has been cancelled,None,None,1701365897\n


In [9]:
# drop duplicates based on a combination of both trip_id and stop_name, while keeping the last occurrence of each combination.
print(f"size before: {orig_size}")
df = df.reset_index().drop_duplicates(subset=['trip_id', 'stop_name'], keep='last').set_index('trip_id')
print(f"size after: {len(df)}")

size before: 16420432
size after: 6335275


In [10]:
df = df.replace({'None': None}, regex=True)
df['realtimeDataUpdatedAt'] = pd.to_datetime(df['realtimeDataUpdatedAt'].str.replace('\n', ''))
df['when'] = pd.to_datetime(df['when'])
df['planned_when'] = pd.to_datetime(df['planned_when'])
df['stop_id'] = df['stop_id'].astype(float)
df['stop_name'] = df['stop_name'].astype(str)
df['line_name'] = df['line_name'].astype(str)
df['start_station'] = df['start_station'].astype(str)
df['end_station'] = df['end_station'].astype(str)
df['end_station_id'] = df['end_station_id'].astype(float)
df['product'] = df['product'].astype(str)
df['occupancy'] = df['occupancy'].astype(str)
df['remark_summary'] = df['remark_summary'].astype(str)
df['remark_start'] = pd.to_datetime(df['remark_start'])
df['remark_end'] = pd.to_datetime(df['remark_end'])
df['delay_seconds'] = df['delay_seconds'].astype(float)


# calculate delay for those where its none
no_delay_mask = df['delay_seconds'].isna()
df.loc[no_delay_mask, 'delay_seconds'] = df.loc[no_delay_mask, 'when'] - df.loc[no_delay_mask, 'planned_when']
df.loc[no_delay_mask, 'delay_seconds'] = df.loc[no_delay_mask, 'delay_seconds'].apply(lambda x: x.total_seconds())

# set information if delay_seconds is calculated
df.loc[no_delay_mask, 'delay_calculated'] = True
df.loc[~no_delay_mask, 'delay_calculated'] = False

df['delay_minutes'] = df['delay_seconds'].apply(lambda x: x / 60)
df = df.join(lat_long, on='stop_id')

/tmp/ipykernel_1140/3894508845.py:14: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['remark_start'] = pd.to_datetime(df['remark_start'])
/tmp/ipykernel_1140/3894508845.py:15: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['remark_end'] = pd.to_datetime(df['remark_end'])
/tmp/ipykernel_1140/3894508845.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '<TimedeltaArray>

In [11]:
df.head()


,stop_id,when,planned_when,stop_name,line_name,delay_seconds,start_station,end_station,end_station_id,product,occupancy,remark_summary,remark_start,remark_end,realtimeDataUpdatedAt,delay_calculated,delay_minutes,latitude,longitude
trip_id,,,,,,,,,,,,,,,,,,,
1|44456|0|86|30112023,900141001.0,2023-11-30 19:03:00+01:00,2023-11-30 19:03:00+01:00,Heinersdorf (Berlin),M2,0.0,Heinersdorf (Berlin),S+U Alexanderplatz Bhf/Dircksenstr. (Berlin),900100024.0,tram,low,None,NaT,NaT,1.701366e+09,False,0.0,52.572342,13.437453
1|44456|0|86|30112023,900141001.0,2023-11-30 19:03:00+01:00,2023-11-30 19:03:00+01:00,Heinersdorf (Berlin),M2,0.0,Heinersdorf (Berlin),S+U Alexanderplatz Bhf/Dircksenstr. (Berlin),900100024.0,tram,low,None,NaT,NaT,1.701366e+09,False,0.0,52.572342,13.437453
1|59997|12|86|30112023,900072181.0,2023-11-30 19:02:00+01:00,2023-11-30 19:02:00+01:00,Großbeerenstr./Daimlerstr. (Berlin),M77,0.0,Großbeerenstr./Daimlerstr. (Berlin),Waldsassener Str. (Berlin),900073281.0,bus,medium,None,NaT,NaT,1.701366e+09,True,0.0,52.425872,13.375643
1|43249|13|86|30112023,900185503.0,2023-11-30 18:56:00+01:00,2023-11-30 18:56:00+01:00,Adlergestell/Vetschauer Allee (Berlin),68,0.0,Adlergestell/Vetschauer Allee (Berlin),Alt--Schmöckwitz (Berlin),900185701.0,tram,low,None,NaT,NaT,1.701365e+09,False,0.0,52.386409,13.633894
1|43249|13|86|30112023,900185503.0,2023-11-30 18:56:00+01:00,2023-11-30 18:56:00+01:00,Adlergestell/Vetschauer Allee (Berlin),68,0.0,Adlergestell/Vetschauer Allee (Berlin),Alt--Schmöckwitz (Berlin),900185701.0,tram,low,None,NaT,NaT,1.701365e+09,False,0.0,52.386409,13.633894


In [12]:
df.to_csv('./delays/final_files/unique_trips_stops.csv')